In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from haversine import  haversine, Unit
from tqdm import tqdm

In [20]:
VERSION = '1.1'

In [2]:
data_path = '../data/'
basic_df = pd.read_csv(data_path+'basic_data.csv')
pharmacy_df = pd.read_csv(data_path+'pharmacy_data.csv')

In [3]:
pharmacy_df

,관리번호,사업장명,도로명주소,전화번호,약국영업면적,인허가일자,영업상태코드,위도,경도
0,PHMD120223060034084000006,연세파란약국,서울특별시 중랑구 동일로 645,02-493-8856,NaN,20221017,1,37.598654,127.079870
1,PHMD120223240033084000019,더조은약국,서울특별시 강동구 상암로 41,02-428-2588,NaN,20221017,1,37.546710,127.150220
2,PHMD120223160034084000006,봄약국,서울특별시 구로구 경인로 204,02-2617-7773,NaN,20220415,1,37.507898,126.886101
3,PHMD120223220033084000026,청담소망약국,서울특별시 강남구 학동로 417,NaN,NaN,20221018,1,37.513421,127.028583
4,PHMD120223120033084000004,참조은약국,서울특별시 서대문구 홍은중앙로8길 20,02-396-3885,NaN,20221018,1,37.598882,126.948222
...,...,...,...,...,...,...,...,...,...
5494,PHMD120223190033084000005,새사랑약국,서울특별시 동작구 서달로 150,NaN,NaN,20220930,1,37.508051,126.963012
5495,PHMD120183180034084000015,맑은수약국,서울특별시 영등포구 여의대방로 65,02-843-2008,NaN,20180423,1,37.521644,126.935753
5496,PHMD120223050034084000009,유원당약국,서울특별시 동대문구 약령중앙로4길 9,02-962-8600,NaN,20221013,1,37.579879,127.037854
5497,PHMD120223050034084000010,보문한약국,서울특별시 동대문구 약령중앙로 11,02-969-0545,NaN,20221013,1,37.580354,127.037881


### 사업장명에서 '약국' 포함하지 않은 명칭이 있는지 확인

In [4]:
name = pharmacy_df['사업장명'].unique()
for n in name:
    if '약국' not in n:
        print(n)

### 도로명 주소 중복 제거

In [5]:
pharmacy_df['도로명주소'].value_counts()

서울특별시 송파구 송파대로 345    12
서울특별시 강남구 일원로 95       9
서울특별시 성동구 왕십리로 410     9
서울특별시 송파구 올림픽로 119     8
서울특별시 종로구 종로 19        6
                      ..
서울특별시 노원구 화랑로 321      1
서울특별시 노원구 덕릉로 779      1
서울특별시 노원구 중계로 120      1
서울특별시 노원구 동일로 1280     1
서울특별시 서초구 강남대로 213     1
Name: 도로명주소, Length: 5024, dtype: int64

In [6]:
pharmacy_df.drop_duplicates(subset='도로명주소', inplace=True)

In [7]:
pharmacy_df

,관리번호,사업장명,도로명주소,전화번호,약국영업면적,인허가일자,영업상태코드,위도,경도
0,PHMD120223060034084000006,연세파란약국,서울특별시 중랑구 동일로 645,02-493-8856,NaN,20221017,1,37.598654,127.079870
1,PHMD120223240033084000019,더조은약국,서울특별시 강동구 상암로 41,02-428-2588,NaN,20221017,1,37.546710,127.150220
2,PHMD120223160034084000006,봄약국,서울특별시 구로구 경인로 204,02-2617-7773,NaN,20220415,1,37.507898,126.886101
3,PHMD120223220033084000026,청담소망약국,서울특별시 강남구 학동로 417,NaN,NaN,20221018,1,37.513421,127.028583
4,PHMD120223120033084000004,참조은약국,서울특별시 서대문구 홍은중앙로8길 20,02-396-3885,NaN,20221018,1,37.598882,126.948222
...,...,...,...,...,...,...,...,...,...
5490,PHMD120203050034084000015,바다약국,서울특별시 동대문구 왕산로 189-1,02-957-1110,NaN,20201103,1,37.580055,127.044136
5492,PHMD120183140033084000009,드림약국,서울특별시 양천구 목동서로 256,02-2654-9503,NaN,20180601,1,37.539034,126.885272
5493,PHMD120223080033084000010,새서울약국,서울특별시 강북구 솔샘로 223,02-981-8215,NaN,20221011,1,37.619466,127.010397
5495,PHMD120183180034084000015,맑은수약국,서울특별시 영등포구 여의대방로 65,02-843-2008,NaN,20180423,1,37.521644,126.935753


### 전화번호

In [8]:
pharmacy_df['전화번호'].isna().sum()

185

In [9]:
pharmacy_df.loc[pharmacy_df[pharmacy_df['전화번호'].isna()==True].index, '전화번호'] = '없음'

In [10]:
pharmacy_df

,관리번호,사업장명,도로명주소,전화번호,약국영업면적,인허가일자,영업상태코드,위도,경도
0,PHMD120223060034084000006,연세파란약국,서울특별시 중랑구 동일로 645,02-493-8856,NaN,20221017,1,37.598654,127.079870
1,PHMD120223240033084000019,더조은약국,서울특별시 강동구 상암로 41,02-428-2588,NaN,20221017,1,37.546710,127.150220
2,PHMD120223160034084000006,봄약국,서울특별시 구로구 경인로 204,02-2617-7773,NaN,20220415,1,37.507898,126.886101
3,PHMD120223220033084000026,청담소망약국,서울특별시 강남구 학동로 417,없음,NaN,20221018,1,37.513421,127.028583
4,PHMD120223120033084000004,참조은약국,서울특별시 서대문구 홍은중앙로8길 20,02-396-3885,NaN,20221018,1,37.598882,126.948222
...,...,...,...,...,...,...,...,...,...
5490,PHMD120203050034084000015,바다약국,서울특별시 동대문구 왕산로 189-1,02-957-1110,NaN,20201103,1,37.580055,127.044136
5492,PHMD120183140033084000009,드림약국,서울특별시 양천구 목동서로 256,02-2654-9503,NaN,20180601,1,37.539034,126.885272
5493,PHMD120223080033084000010,새서울약국,서울특별시 강북구 솔샘로 223,02-981-8215,NaN,20221011,1,37.619466,127.010397
5495,PHMD120183180034084000015,맑은수약국,서울특별시 영등포구 여의대방로 65,02-843-2008,NaN,20180423,1,37.521644,126.935753


In [11]:
pharmacy_df['전화번호'].value_counts()[:20]

없음              185
02-000-0000       7
02-0000-0000      6
02-               5
02                3
02-3463-7510      2
020000000         2
000-0000          2
02-373-9725       2
0                 2
-                 2
02-825-4532       1
02-822-0667       1
02-817-6087       1
02-815-3011       1
02-583-2882       1
02-826-8347       1
02-848-4566       1
02-823-0583       1
02-823-8336       1
Name: 전화번호, dtype: int64

In [12]:
phone = pharmacy_df['전화번호'].unique()
wrong = ['02-000-0000','02-0000-0000', '02-', '02', '020000000', '000-0000', '0', '-']
for p in phone:
    if p in wrong:
        pharmacy_df.loc[pharmacy_df[pharmacy_df['전화번호']==p].index, '전화번호'] = '없음'

In [13]:
pharmacy_df['전화번호'].value_counts()[:20]

없음              214
02-3463-7510      2
02-373-9725       2
02-493-8856       1
02-582-5353       1
02-822-0667       1
02-817-6087       1
02-815-3011       1
02-583-2882       1
02-826-8347       1
02-823-0583       1
02-814-1943       1
02-815-0506       1
02-821-3157       1
02-822-5692       1
02-831-1922       1
02-824-9447       1
02-817-8303       1
02-535-4674       1
02-522-0993       1
Name: 전화번호, dtype: int64

### 약국영업면적

In [14]:
pharmacy_df['약국영업면적'].value_counts()[:20]

0.0     219
33.0     19
25.0     18
22.0     17
23.0     17
30.0     16
24.0     16
21.0     16
28.0     15
29.0     14
32.0     14
28.8     14
35.0     13
54.0     13
40.0     13
16.0     12
48.0     12
36.0     12
20.0     12
Name: 약국영업면적, dtype: int64

In [15]:
pharmacy_df['약국영업면적'].isna().sum()

512

In [16]:
pharmacy_df.loc[pharmacy_df[pharmacy_df['약국영업면적'].isna()==True].index, '약국영업면적'] = '없음'
pharmacy_df.loc[pharmacy_df[pharmacy_df['약국영업면적']==0.0].index, '약국영업면적'] = '없음'

In [17]:
pharmacy_df['약국영업면적'].value_counts()[:20]

없음      731
33.0     19
25.0     18
23.0     17
22.0     17
21.0     16
30.0     16
24.0     16
28.0     15
28.8     14
29.0     14
32.0     14
54.0     13
40.0     13
35.0     13
48.0     12
36.0     12
16.0     12
20.0     12
25.2     11
Name: 약국영업면적, dtype: int64

### 인허가일자

In [16]:
pharmacy_df['인허가일자'].isna().sum()

0

In [19]:
pharmacy_df['인허가일자'].value_counts()

20190701    17
20170102    15
20190304    14
20191202    13
20200302     9
            ..
20151002     1
20060214     1
20120817     1
20140116     1
20221011     1
Name: 인허가일자, Length: 3391, dtype: int64

### 영업상태코드

In [18]:
pharmacy_df['영업상태코드'].unique()

array([1], dtype=int64)

### 위도 경도

- 나눠져 있는거 합치기

In [20]:
pharmacy_df['위도'].isna().sum(), pharmacy_df['경도'].isna().sum()

(0, 0)

In [36]:
loc = pharmacy_df['도로명주소'].unique()
pharmacy_df['location'] = 0
for l in loc:
    latitude = pharmacy_df[pharmacy_df['도로명주소']==l]['위도'].values
    longitude = pharmacy_df[pharmacy_df['도로명주소']==l]['경도'].values
    location = str(latitude[0]) + ',' + str(longitude[0])
    pharmacy_df.loc[pharmacy_df[pharmacy_df['도로명주소']==l].index, 'location'] = location

In [43]:
pharmacy_df.drop(columns=['위도','경도'], axis=1, inplace=True)
pharmacy_df

,관리번호,사업장명,도로명주소,전화번호,약국영업면적,인허가일자,영업상태코드,location
0,PHMD120223060034084000006,연세파란약국,서울특별시 중랑구 동일로 645,02-493-8856,없음,20221017,1,"37.5986537,127.0798699"
1,PHMD120223240033084000019,더조은약국,서울특별시 강동구 상암로 41,02-428-2588,없음,20221017,1,"37.54671,127.15022"
2,PHMD120223160034084000006,봄약국,서울특별시 구로구 경인로 204,02-2617-7773,없음,20220415,1,"37.50789805,126.8861006666827"
3,PHMD120223220033084000026,청담소망약국,서울특별시 강남구 학동로 417,없음,없음,20221018,1,"37.5134213,127.0285826"
4,PHMD120223120033084000004,참조은약국,서울특별시 서대문구 홍은중앙로8길 20,02-396-3885,없음,20221018,1,"37.5988819,126.9482225"
...,...,...,...,...,...,...,...,...
5490,PHMD120203050034084000015,바다약국,서울특별시 동대문구 왕산로 189-1,02-957-1110,없음,20201103,1,"37.58005485,127.0441356900642"
5492,PHMD120183140033084000009,드림약국,서울특별시 양천구 목동서로 256,02-2654-9503,없음,20180601,1,"37.5390345,126.8852717"
5493,PHMD120223080033084000010,새서울약국,서울특별시 강북구 솔샘로 223,02-981-8215,없음,20221011,1,"37.6194664,127.0103967"
5495,PHMD120183180034084000015,맑은수약국,서울특별시 영등포구 여의대방로 65,02-843-2008,없음,20180423,1,"37.5216437,126.9357534"


### 저장

In [44]:
pharmacy_df.to_csv(data_path+f'pharmacy_data_{VERSION}.csv', index=False)